In [1]:
import os
import glob
import random 
import itertools

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
from scipy import signal
from datetime import datetime

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader, ConcatDataset, random_split, Subset

from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
label_encoder = preprocessing.LabelEncoder()

In [2]:
import wandb
# WandBの設定
WANDB_API_KEY = "2d996a98ef8dddefa91d675f85b5efd96fb911ae"  # あなたのWandB APIキーをここに入力してください

wandb.login(key = WANDB_API_KEY)

wandb: Currently logged in as: nekodaisuki169 (doctor_thesis_material). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/sota/.netrc


True

In [3]:


class TactileSequenceDataset(Dataset):
    """
    触覚センサ npy データ用 Dataset クラス

    - 入力ファイル: shape (T, n_taxels, 3)
    - seq_start〜seq_end ステップを切り出し
    - (seq_len, n_taxels, 3) -> (seq_len, n_taxels*3) にフラット化
    - ESN 用に (1, seq_len, feature_dim) に reshape し、バッチ化で
      (batch_size, 1, seq_len, feature_dim) になるようにする
    """

    def __init__(self, root_dir, dataset_params):
        """
        Args:
            root_dir: クラスディレクトリが並んでいるルートパス
                      例: dataset_root/
                            paper_A4/
                              xxx.npy
                            cloth/
                              yyy.npy
            dataset_params:
                - "seq_start": 切り出し開始インデックス（含む）
                - "seq_end"  : 切り出し終了インデックス（含まない）
        """
        super().__init__()
        self.root_dir = Path(root_dir)
        self.seq_start = int(dataset_params["seq_start"])
        self.seq_end   = int(dataset_params["seq_end"])
        self.dtype     = torch.float32  # ← カンマ削除

        # クラスディレクトリを列挙
        self.class_names = sorted(
            [d.name for d in self.root_dir.iterdir() if d.is_dir()]
        )
        if not self.class_names:
            raise RuntimeError(f"No class directories under {self.root_dir}")

        self.class_to_idx = {c: i for i, c in enumerate(self.class_names)}
        self.num_classes  = len(self.class_names)  # ★ one-hot 用

        # 各 npy ファイルのパスとラベルを列挙
        self.samples = []  # list of (path, label_idx)
        for class_name in self.class_names:
            class_dir = self.root_dir / class_name
            for npy_path in sorted(class_dir.glob("*.npy")):
                self.samples.append((npy_path, self.class_to_idx[class_name]))

        if not self.samples:
            raise RuntimeError(f"No .npy files found under {self.root_dir}")

        # シーケンス長を確認 (最初のサンプルでチェック)
        arr0 = np.load(self.samples[0][0])
        T, n_taxels, axes = arr0.shape
        assert axes == 3, f"Last dimension must be 3 (x,y,z), got {axes}"
        if self.seq_end > T:
            raise ValueError(
                f"seq_end({self.seq_end}) is larger than T({T}). "
                "Please adjust seq_start/seq_end."
            )

        self.original_T = T
        self.n_taxels   = n_taxels
        self.seq_len    = self.seq_end - self.seq_start
        self.feature_dim = self.n_taxels * 3  # x, y, z をまとめた次元

        print("=== TactileSequenceDataset initialized ===")
        print(f"root_dir     : {self.root_dir}")
        print(f"num_classes  : {self.num_classes}")
        print(f"num_samples  : {len(self.samples)}")
        print(f"original T   : {self.original_T}")
        print(f"seq range    : [{self.seq_start}, {self.seq_end}) -> seq_len={self.seq_len}")
        print(f"n_taxels     : {self.n_taxels}")
        print(f"feature_dim  : {self.feature_dim} (= n_taxels * 3)")
        print("class_to_idx : ", self.class_to_idx)
        print("=========================================")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        npy_path, label_idx = self.samples[idx]
        arr = np.load(npy_path)  # shape (T, n_taxels, 3)

        # 指定区間を切り出し
        seq = arr[self.seq_start:self.seq_end]  # (seq_len, n_taxels, 3)

        # (seq_len, n_taxels, 3) -> (seq_len, n_taxels * 3)
        seq = seq.reshape(self.seq_len, self.feature_dim)

        # torch.Tensor に変換 (seq_len, feature_dim)
        x = torch.from_numpy(seq).to(self.dtype)

        # ESN が期待する (batch, channel_size, seq_len, input_size) に合わせて
        # channel_size = 1 として (1, seq_len, feature_dim) に変換
        x = x.unsqueeze(0)  # (1, seq_len, feature_dim)

        # ラベルを one-hot ベクトルに変換
        label_idx_tensor = torch.tensor(label_idx, dtype=torch.long)
        y = F.one_hot(label_idx_tensor, num_classes=self.num_classes).to(self.dtype)
        # y.shape -> (num_classes,)  バッチ化後: (batch_size, num_classes)

        return x, y


In [4]:
class EchoStateNetwork(nn.Module):
    def __init__(self, model_params, dataset_params):
        super(EchoStateNetwork, self).__init__()
        
        self.reservoir_size = int(model_params["reservoir_size"])
        self.reservoir_weights_scale = float(model_params["reservoir_weights_scale"])
        
        self.input_size = int(model_params["input_size"])
        self.channel_size = int(model_params["channel_size"])
        self.input_weights_scale = float(model_params["input_weights_scale"])
        self.spectral_radius = float(model_params["spectral_radius"])
        self.density = float(model_params["reservoir_density"])
        self.leak_rate = float(model_params["leak_rate"])
        
        self.sequence_length = int(int(dataset_params["sequence_length"]) / int(dataset_params["slicing_size"]))
        # self.sequence_length = int(dataset_params["seq_end"]) - int(dataset_params["seq_start"])

        # リザバー結合行列 (ランダムに初期化)
        self.register_parameter("reservoir_weights", torch.nn.Parameter(torch.empty((self.reservoir_size, self.reservoir_size)).uniform_(-self.reservoir_weights_scale, self.reservoir_weights_scale).to(device), requires_grad=False))
        
        # リザバー入力行列 (ランダムに初期化)
        self.register_parameter("input_weights", torch.nn.Parameter(torch.empty((self.reservoir_size, self.input_size * self.channel_size)).uniform_(-self.input_weights_scale, self.input_weights_scale).to(device), requires_grad=False))
        
        

        #リザバー結合のスパース処理
        self.reservoir_weights_mask = torch.empty((self.reservoir_size, self.reservoir_size)).uniform_(0, 1)
        self.reservoir_weights_mask = torch.where(self.reservoir_weights_mask < self.density, torch.tensor(1), torch.tensor(0)).to(device)
        self.reservoir_weights *= self.reservoir_weights_mask
        
        #スペクトル半径の処理
        _, singular_values, _ = torch.svd(self.reservoir_weights)
        rho_reservoir_weights = torch.max(singular_values).item()
        self.reservoir_weights *= self.spectral_radius / rho_reservoir_weights
        
       #最終時刻における，リザバー状態ベクトル
        self.last_reservoir_state_matrix = torch.zeros(self.channel_size, self.reservoir_size).to(device)
    

    def forward(self, x):
        # print(x.shape)
        batch_size = x.size(0)
        sequence_length = x.size(2)
        # x_seq = x.view(batch_size, self.channel_size, self.input_size, sequence_length).to(device)

        # if self.last_reservoir_state_matrix is None or self.last_reservoir_state_matrix.size(0) != batch_size:
        #     self.last_reservoir_state_matrix = torch.zeros(
        #         batch_size, self.channel_size, self.reservoir_size, device=device
        #     )
        
        # 各シーケンスのバッチに対してリザバー状態を初期化
        self.reservoir_state_matrix = torch.zeros(batch_size, self.channel_size,  sequence_length, self.reservoir_size).to(device)

        self.last_reservoir_state_matrix = torch.zeros(
                batch_size, self.channel_size, self.reservoir_size, device=device
            )
        
        for t in range(sequence_length):
            input_at_t = torch.matmul(x[:, :, t, :].view(batch_size, -1), self.input_weights.t())
            input_at_t = input_at_t.unsqueeze(1)

            if t == 0:
                state_update = torch.matmul(self.last_reservoir_state_matrix, self.reservoir_weights)
            else:
                # print("11111")
                state_update = torch.matmul(self.reservoir_state_matrix[:, :, t-1, :], self.reservoir_weights)
                # print(f"state_update.shape {state_update.shape}")
            self.reservoir_state_matrix[:, :, t, :] = self.leak_rate * torch.tanh(input_at_t + state_update) + \
                                                    (1 - self.leak_rate) * self.reservoir_state_matrix[:, :, t-1, :]

        

        self.last_reservoir_state_matrix = self.reservoir_state_matrix[:, :, -1, :]
        return self.reservoir_state_matrix

    def reset_hidden_state(self):
        self.last_reservoir_state_matrix = torch.zeros(self.channel_size, self.reservoir_size, device=device)
        # pass
        # print("内部状態がリセットされました")
    
#リードアウト層
class ReadOut(nn.Module):
    def __init__(self, model_params, dataset_params):
        super(ReadOut, self).__init__()
        # self.reservoir_state_matrix_size = int(model_params["reservoir_size"]) + int(model_params["input_size"]) + 1
        self.reservoir_state_matrix_size = int(model_params["reservoir_size"])
        self.output_size = int(model_params["ReadOut_output_size"])
        self.batch_training = model_params["Batch_Training"]
        self.channel_size = int(model_params["channel_size"])
        
        self.sequence_length = int(int(dataset_params["sequence_length"]) / int(dataset_params["slicing_size"]))
        # self.sequence_length = int(dataset_params["seq_end"]) - int(dataset_params["seq_start"])
        
        self.readout_dense = nn.Linear(self.reservoir_state_matrix_size, self.output_size, bias=False)
        
        #線形回帰におけるバッチ学習を行うならば，リードアウト層を最急降下法による学習対象にしない
        if self.batch_training == True:
            self.readout_dense.weight.requires_grad = False
        else:
            None
            
        nn.init.xavier_uniform_(self.readout_dense.weight)
        
    def forward(self, x):
        # x shape: [batch_size, channel_size, input_size, sequence_length]
        # batch_size, channel_size, input_size, sequence_length = x.size()
        
        # Reshape x to apply the dense layer to each time step and channel independently
        # New shape: [batch_size * channel_size * sequence_length, input_size]
        # x_reshaped = x.permute(0, 1, 3, 2).contiguous().view(-1, input_size)
        
        # Apply the dense layer
        # Output shape: [batch_size * channel_size * sequence_length, output_size]
        # print(f"readout x shape{x.shape}")
        output = self.readout_dense(x)
        
        # Reshape the output back to the original form
        # Output shape: [batch_size, channel_size, output_size, sequence_length]
        # output = output.view(batch_size, channel_size, sequence_length, -1).permute(0, 1, 3, 2).contiguous()
        return output
    
    # リッジ回帰によるリードアウトの導出
    @staticmethod
    def ridge_regression(X, Y, alpha):
        # データ行列 X の形状を取得
        n, p = X.shape

        # 正則化項の行列を作成
        ridge_matrix = (alpha * torch.eye(n)).float().to(device)
        X = X.float()
        Y = Y.float()

        # リッジ回帰の係数を計算
        coefficients = torch.linalg.solve(torch.matmul(X, X.T) + ridge_matrix, torch.matmul(X, Y.T)).T

        return coefficients

    @staticmethod
    def ridge_regression_update(outputs, targets, model, alpha=0):
        with torch.no_grad():
            # リッジ回帰を用いて重みを求める
            # モデルのパラメータを更新
            new_weights = ReadOut.ridge_regression(outputs.squeeze(), targets.squeeze(), alpha)

            # モデルのパラメータを更新
            model.ReadOut.readout_dense.weight.copy_(new_weights)
        
        return None
    #それぞれのモデルパラメータ候補を辞書に格納する
    @staticmethod
    def model_params_candinate(model_params):
        model_params_combinations = list(itertools.product(*model_params.values()))
        param_dicts = [dict(zip(model_params.keys(), combination)) for combination in model_params_combinations]
        
        return param_dicts

    #モデル構造を辞書型に格納
    @staticmethod
    def model_sturcture_dict(model):
        layers_dict = {}
        for name, module in model.named_modules():
            layers_dict[name] = {
                'type': type(module).__name__,
                'parameters': {p: getattr(module, p) for p in module.__dict__ if not p.startswith('_')}
            }
        
        #モデル名と初期の引数は削除
        del(layers_dict[''])
        
        return layers_dict

class ESN(nn.Module):
    def __init__(self, model_params, training_params, dataset_params):
        super(ESN, self).__init__()
        self.ESN = EchoStateNetwork(model_params, dataset_params)
        self.ReadOut = ReadOut(model_params, dataset_params)
    
    def forward(self, x):
        self.Reservoir_State_Matrix = self.ESN(x)
        # print(f"resever state matrix {self.Reservoir_State_Matrix.shape}")
        self.ReadOut_Reservoir = self.ReadOut(self.Reservoir_State_Matrix)
        # print("重み:", self.ReadOut.readout_dense.weight)
        # print(self.ReadOut_Reservoir)

        #channle_size次元はいらないので，減らす
        self.ReadOut_Reservoir = self.ReadOut_Reservoir.squeeze(1)
        # print(f"output matrix {self.ReadOut_Reservoir.shape}")

        return self.ReadOut_Reservoir

    def reset_hidden_state(self):
        self.ESN.reset_hidden_state()

In [5]:
# K分割交差検証とデータローダーの生成
def create_cross_validation_dataloaders(dataset, dataset_params, traing_params):
    n_splits = int(traing_params["n_splits"])
    batch_size = int(dataset_params["batch_size"])
    augmentation_factor = int(dataset_params["augmentation_factor"])
    
    kf = KFold(n_splits=n_splits)
    split_datasets = []

    for train_idx, val_idx in kf.split(range(len(dataset))):
        train_subset = Subset(dataset, train_idx)
        val_subset = Subset(dataset, val_idx)
        
        # 学習用サブセットにのみ拡張を適用
        # train_subset = AugmentedDataset(train_subset, dataset_params)

        train_loader = DataLoader(train_subset, len(train_subset), shuffle=True)
        val_loader = DataLoader(val_subset, len(val_subset), shuffle=False)

        split_datasets.append((train_loader, val_loader))
    
    return split_datasets

def prepare_datasets(dataset_params, traing_params, data_dir):
    testdata_ratio = float(traing_params["testdata_ratio"])
    batch_size = int(dataset_params["batch_size"])
    
    # データセットの準備
    dataset = TactileSequenceDataset(data_dir, dataset_params)  # 拡張はここでは適用しない

    # データセットを学習用、テスト用に分割する
    test_size = int(len(dataset) * testdata_ratio)
    train_size = len(dataset) - test_size
    crossval_dataset, test_dataset = random_split(dataset, [train_size, test_size])

    # 交差検証のためのデータローダーを準備
    cross_val_loaders = create_cross_validation_dataloaders(crossval_dataset, dataset_params, traing_params)

    # テストデータローダーの準備
    test_loader = DataLoader(test_dataset, batch_size, shuffle=False)

    return cross_val_loaders, test_loader

In [6]:
def train_model(model, criterion, optimizer, train_loader, dataset_params, model_params, traing_params):
    model.train()  # Set model to training mode
    num_epochs = int(traing_params["num_epochs"])   
    reservoir_size = int(model_params["reservoir_size"])   
    input_size = int(model_params["input_size"])
    ReadOut_output_size = int(model_params["ReadOut_output_size"])
    sequence_length = int(int(dataset_params["sequence_length"]) / int(dataset_params["slicing_size"]))
    batch_training = model_params["Batch_Training"]
    Regularization_L2 = model_params["Regularization_L2"]

    for epoch in range(num_epochs):
        running_loss = 0.
        for inputs, labels in train_loader:
            inputs = inputs.float().to(device)
            # print(inputs.shape)
                
            labels = labels.to(device)
            
            optimizer.zero_grad()

            outputs_ESN = model.ESN(inputs)
            outputs = model.ReadOut(outputs_ESN)
            outputs = outputs.squeeze()
            
            B, T, C = outputs.shape                # B=200, T=800, C=25

            # ---- flattenの整合性を取る ----
            # リザバー： [500, B*T]
            outputs_ESN_flatten = outputs_ESN.view(reservoir_size, -1)
        
            # 出力： [C, B*T]
            outputs_flatten = outputs.view(C, -1)
        
            # ラベル：各サンプルの one-hot を T 回繰り返して [B, T, C] → [C, B*T]
            labels_rep = labels.unsqueeze(1).repeat(1, T, 1)  # [B, T, C]
            labels_flatten = labels_rep.view(C, -1)           # [C, B*T]
            # print(outputs_flatten.shape)
            # print(labels_flatten.shape)
        
            loss = criterion(outputs_flatten, labels_flatten)
            
            if batch_training == True:
                model.ReadOut.ridge_regression_update(outputs_ESN_flatten, labels_flatten, model, Regularization_L2)
            else:
                #勾配の計算
                loss.backward()
                #重みの更新
                optimizer.step()
            
            running_loss += loss.item() 

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')
        wandb.log({"loss": epoch_loss})
    
    print('Training complete')
    
def validate_model(model, val_loader, dataset_params, model_params):
    model.eval()  # Set model to evaluation mode
    val_running_loss = 0.0
    val_running_corrects = 0
    
    reservoir_size = int(model_params["reservoir_size"])   
    input_size = int(model_params["input_size"])
    ReadOut_output_size = int(model_params["ReadOut_output_size"])
    sequence_length = int(int(dataset_params["sequence_length"]) / int(dataset_params["slicing_size"]))

    with torch.no_grad():
        for inputs, labels in val_loader:
            # print(inputs.shape)
            inputs = inputs.float().to(device)
            labels = labels.to(device)
            outputs_ESN = model.ESN(inputs)      # [B, 1, T, 500]
            outputs = model.ReadOut(outputs_ESN).squeeze(1)  # [B, T, C]

            B, T, C = outputs.shape

            outputs_flatten = outputs.view(C, -1)
            labels_rep = labels.unsqueeze(1).repeat(1, T, 1)
            labels_flatten = labels_rep.view(C, -1)

            loss = criterion(outputs_flatten, labels_flatten)
            val_running_loss += loss.item() * B

    val_loss = val_running_loss / len(val_loader.dataset)
    val_accuracy = compute_accuracy(outputs_flatten, labels_flatten, sequence_length)
    print(f"Accuracy: {val_accuracy * 100:.2f}%")
    
    print(f'Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}')
    wandb.log({"val_accuracy": val_accuracy, "val_loss": val_loss})
    
def test_model(model, val_loader):
    model.eval()  # Set model to evaluation mode
    val_running_loss = 0.0
    val_running_corrects = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.float().to(device)
            labels = labels.squeeze().to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)
#             _, label_preds = torch.max(labels, 1)
            label_preds = labels

            val_running_loss += loss.item() * inputs.size(0)
            print(loss.item())
            val_running_corrects += torch.sum(preds == label_preds)

    val_loss = val_running_loss / len(val_loader.dataset)
    val_accuracy = val_running_corrects.double() / len(val_loader.dataset)
    
    print(f'Test Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}')
    wandb.log({"test_accuracy": val_accuracy, "test_loss": val_loss})

def compute_accuracy(model_output, target, n_taus):
    # モデルの出力と教師ラベルを各データに分割
    split_model_output = torch.split(model_output.squeeze(), n_taus, dim=-1)
    split_target = torch.split(target.squeeze(), n_taus, dim=-1)
    # print("aaaa")
    # print(split_model_output[0].shape)
    # print(split_target[0].shape)
    correct = 0
    total = 0
    
    for pred, true_label in zip(split_model_output, split_target):
        # 最も確率が高いラベルを予測ラベルとして取得
        # print(pred.shape)
        # print(pred)
        # print(true_label.shape)
        # print(true_label)
        count_ones = (true_label == 1).sum().item()
        # print(count_ones)
        histgram_predict = torch.bincount(torch.max(pred, 0)[1])
        _, predicted = torch.max(histgram_predict, 0)
    
        histgram_true_label_idx = torch.bincount(torch.max(true_label, 0)[1])
        _, true_label_idx = torch.max(histgram_true_label_idx, 0)

        # 正解数をカウント
        correct += (predicted == true_label_idx).sum().item()
        total += 1

    # 精度を算出
    accuracy = correct / total
    return accuracy

def model_params_candinate(model_params):
    model_params_combinations = list(itertools.product(*model_params.values()))
    param_dicts = [dict(zip(model_params.keys(), combination)) for combination in model_params_combinations]
    return param_dicts

# モデル構造を辞書型に格納
def model_sturcture_dict(model):
    layers_dict = {}
    for name, module in model.named_modules():
        layers_dict[name] = {
            'type': type(module).__name__,
            'parameters': {p: getattr(module, p) for p in module.__dict__ if not p.startswith('_')}
        }
    # モデル名と初期の引数は削除
    del(layers_dict[''])
    return layers_dict
    

In [7]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime
import json
import hashlib
import wandb

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def visualize_reservoir_states(
    model,
    train_loader,
    dataset_params,
    model_params,
    traing_params,
    n_units_to_plot=None,     # 線グラフに出すユニット数（None=全ノード）
    units_for_heatmap=None,   # ヒートマップに含めるユニット集合（None=全ノード）
    save_root="reservoir_vis",# 実行全体のルート
    run_dir=None,             # 実行開始日時分ディレクトリ（None のときは自動生成）
):
    """
    学習は行わず、ESN のリザバー状態を可視化 & 統計指標計算する関数。

    ディレクトリ構造:
      save_root/
        YYYYMMDD_HHMM/             ← 実行開始日時分 run_dir
          hyperparams_index.json   ← ハイパラID → (params + metrics) の対応表
          hp_xxxxxxxx/             ← 各ハイパラ構成
            config.json            ← dataset/model/training + metrics 一式
            class_00_xxx.png       ← クラスごとの可視化結果
            class_01_xxx.png
            ...

    run_dir を外側で固定して渡すと、全ハイパラが同じ日時ディレクトリに入る。
    """

    model.eval()

    # ===== Dataset 情報の取得 =====
    base_dataset = getattr(train_loader, "dataset", None)
    if base_dataset is None:
        raise ValueError("train_loader.dataset が存在しません。")

    # 一番奥の元データセット（TactileSequenceDataset）を取り出す
    root_dataset = base_dataset
    while hasattr(root_dataset, "dataset"):
        root_dataset = root_dataset.dataset

    class_names = getattr(root_dataset, "class_names", None)

    # ===== 1クラスにつき1サンプルのインデックスを収集 =====
    class_to_sample_idx = {}  # key: class_idx -> sample_idx (base_dataset 上の index)

    for idx in range(len(base_dataset)):
        _, y = base_dataset[idx]  # y: one-hot (num_classes,) を想定
        if y.dim() > 1:
            y_flat = y.view(-1)
        else:
            y_flat = y
        class_idx = int(y_flat.argmax().item())

        if class_idx not in class_to_sample_idx:
            class_to_sample_idx[class_idx] = idx

    available_classes = sorted(class_to_sample_idx.keys())
    print(f"Found representative samples for {len(available_classes)} classes (class indices: {available_classes})")

    # ===== 実行開始日時分ディレクトリ（run_dir）を決める =====
    if run_dir is None:
        # 例: 20251215_0131 （YYYYMMDD_HHMM）
        timestamp_dirname = datetime.now().strftime("%Y%m%d_%H%M")
        run_dir = Path(save_root) / timestamp_dirname
    else:
        run_dir = Path(run_dir)
        timestamp_dirname = run_dir.name  # wandb の key 用に使う

    run_dir.mkdir(parents=True, exist_ok=True)
    print(f"Run directory: {run_dir}")

    # ===== ハイパラ構成から短いIDディレクトリ名を生成 =====
    # dataset_params / model_params / traing_params をまとめて json → MD5 → 先頭8文字
    hp_config = {
        "dataset_params": dataset_params,
        "model_params": model_params,
        "training_params": traing_params,
    }
    hp_json_base = json.dumps(hp_config, sort_keys=True, ensure_ascii=False, indent=2)
    hp_hash = hashlib.md5(hp_json_base.encode("utf-8")).hexdigest()[:8]
    hp_dirname = f"hp_{hp_hash}"

    # 実行開始日時分 / 短いハイパラID の 2段階ディレクトリ
    hp_dir = run_dir / hp_dirname
    hp_dir.mkdir(parents=True, exist_ok=True)
    print(f"Saving visualizations under: {hp_dir}")

    # ===== クラスごとの states を貯めて、最後にハイパラ単位で指標計算する =====
    sample_states_list = []  # 各クラス代表サンプルの (T, H) を入れていく

    # ===== クラスごとに1サンプルずつ可視化 =====
    for class_idx in available_classes:
        sample_idx = class_to_sample_idx[class_idx]
        x, y = base_dataset[sample_idx]  # x: (1, T, F), y: (num_classes,)

        # ESN 入力形状に合わせて (1, 1, T, F) に
        inputs = x.unsqueeze(0).to(device)   # (1, 1, T, F)

        with torch.no_grad():
            outputs_ESN = model.ESN(inputs)  # 形状: (1, 1, T, H) or (1, T, H)

        # 形状を (T, H) に変換
        if outputs_ESN.dim() == 4:
            states = outputs_ESN[0, 0].detach().cpu().numpy()
        elif outputs_ESN.dim() == 3:
            states = outputs_ESN[0].detach().cpu().numpy()
        else:
            raise ValueError(f"Unexpected ESN output shape for single sample: {outputs_ESN.shape}")

        sample_states_list.append(states)  # ハイパラ指標計算用に保持

        T_steps, H_units = states.shape
        t = np.arange(T_steps)

        # クラス名（あれば）
        if class_names is not None and 0 <= class_idx < len(class_names):
            class_name = class_names[class_idx]
        else:
            class_name = f"class_{class_idx}"

        # 線グラフ対象ユニット数
        if n_units_to_plot is None:
            n_plot_units = H_units
        else:
            n_plot_units = min(n_units_to_plot, H_units)

        # ヒートマップ対象ユニット（None → 全ノード）
        if units_for_heatmap is None:
            unit_indices = np.arange(H_units)
        elif isinstance(units_for_heatmap, int):
            N = min(units_for_heatmap, H_units)
            unit_indices = np.arange(N)
        else:
            unit_indices = np.array(
                [u for u in units_for_heatmap if 0 <= u < H_units],
                dtype=int
            )

        # ===== 図の作成：上段 = 全ノードの軌跡, 下段 = ノード×時間ヒートマップ =====
        fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=False)

        # ---- 上段：ユニットごとの軌跡（全ノード or 指定数）----
        ax_line = axes[0]
        for u in range(n_plot_units):
            ax_line.plot(t, states[:, u], alpha=0.7)

        ax_line.set_xlabel("time step")
        ax_line.set_ylabel("state value")
        ax_line.set_title(
            f"Reservoir trajectories | class={class_idx}: {class_name} "
            f"(sample_idx={sample_idx})"
        )
        ax_line.grid(True)   # 凡例なし

        # ---- 下段：各ノードが取る値の時間変化（横:時間, 縦:ノード）----
        # states: (T, H) → (N_units, T) にして imshow → 横:時間, 縦:ノード
        heat_states = states[:, unit_indices].T  # (N_units, T)

        ax_img = axes[1]
        im = ax_img.imshow(
            heat_states,
            aspect="auto",
            origin="lower",   # unit index 0 を下に
            cmap="viridis"
        )

        ax_img.set_xlabel("time step")
        ax_img.set_ylabel("unit index")

        # y 軸にユニット index を表示（多い場合は適宜間引き）
        ax_img.set_yticks(
            np.linspace(0, len(unit_indices) - 1, min(10, len(unit_indices)))
        )
        ytick_labels = [str(unit_indices[int(idx)]) for idx in ax_img.get_yticks()]
        ax_img.set_yticklabels(ytick_labels)

        ax_img.set_title(
            f"Reservoir states (units × time)\n"
            f"class={class_idx}: {class_name} (sample_idx={sample_idx})"
        )

        fig.colorbar(im, ax=ax_img, label="state value")

        plt.tight_layout()

        # ===== ファイル保存 =====
        safe_class_name = str(class_name).replace("/", "_")
        filename = f"class_{class_idx:02d}_{safe_class_name}.png"
        save_path = hp_dir / filename
        fig.savefig(save_path, dpi=150)

        # ===== wandb にも同じ構造で保存（run_dir名/ハイパラID/ファイル） =====
        wandb_key = f"{timestamp_dirname}/{hp_dirname}/{filename}"
        wandb.log({wandb_key: wandb.Image(fig)})

        plt.close(fig)

        print(
            f"[class {class_idx}] {class_name} | sample_idx={sample_idx} | "
            f"states shape={states.shape} | saved to {save_path}"
        )

    # ===== ハイパラ単位でリザバーの「豊かさ」指標を計算 =====
    metrics = {}
    if len(sample_states_list) > 0:
        all_states = np.vstack(sample_states_list)  # (T_total, H)
        T_total, H_units = all_states.shape

        # 1. 各ノードの時間方向分散
        temporal_var_per_node = np.var(all_states, axis=0)  # (H,)
        mean_temporal_variance = float(temporal_var_per_node.mean())
        dead_eps = 1e-4
        dead_unit_ratio = float((temporal_var_per_node < dead_eps).mean())

        # 2. dynamic range
        dynamic_range_per_node = all_states.max(axis=0) - all_states.min(axis=0)
        mean_dynamic_range = float(dynamic_range_per_node.mean())
        small_range_delta = 1e-3
        small_dynamic_range_ratio = float((dynamic_range_per_node < small_range_delta).mean())

        # 3. 各時刻でのノード方向分散
        node_var_per_time = np.var(all_states, axis=1)  # (T_total,)
        mean_node_variance_across_nodes = float(node_var_per_time.mean())

        # 4. 状態ベクトルノルム
        state_norms = np.linalg.norm(all_states, axis=1)
        mean_state_norm = float(state_norms.mean())
        std_state_norm = float(state_norms.std())

        # 5. 飽和率 (|state| > 0.9)
        tau = 0.9
        saturation_ratio = float((np.abs(all_states) > tau).mean())

        # 6. 共分散 & participation ratio
        cov = np.cov(all_states, rowvar=False)  # (H, H)
        if np.allclose(cov, 0):
            participation_ratio = 0.0
        else:
            eigvals = np.linalg.eigvalsh(cov)
            eigvals = np.clip(eigvals, a_min=0.0, a_max=None)
            sum_eig = float(eigvals.sum())
            sum_sq = float((eigvals ** 2).sum())
            participation_ratio = float((sum_eig ** 2 / sum_sq) if sum_sq > 0 else 0.0)

        # 7. ノード間相関の平均絶対値（対角除く）
        eps = 1e-12
        X = all_states - all_states.mean(axis=0, keepdims=True)
        std = all_states.std(axis=0, ddof=0)
        std_safe = np.where(std < eps, 1.0, std)
        Xn = X / std_safe  # 各列を標準化
        # 相関行列 ≒ 標準化後の共分散
        n_samples_t = Xn.shape[0]
        corr = np.dot(Xn.T, Xn) / max(n_samples_t - 1, 1)  # (H, H)

        off_diag = corr - np.eye(H_units)
        mean_abs_correlation = float(np.sum(np.abs(off_diag)) / (H_units * (H_units - 1)))

        metrics = {
            "num_samples": int(len(sample_states_list)),
            "num_units": int(H_units),
            "total_time_steps": int(T_total),
            "mean_temporal_variance": mean_temporal_variance,
            "dead_unit_ratio": dead_unit_ratio,
            "mean_dynamic_range": mean_dynamic_range,
            "small_dynamic_range_ratio": small_dynamic_range_ratio,
            "mean_node_variance_across_nodes": mean_node_variance_across_nodes,
            "mean_state_norm": mean_state_norm,
            "std_state_norm": std_state_norm,
            "saturation_ratio": saturation_ratio,
            "mean_abs_correlation": mean_abs_correlation,
            "participation_ratio": participation_ratio,
        }

        print("Reservoir richness metrics:", metrics)

        # wandb にもハイパラ単位でログ
        wandb.log({f"{hp_dirname}/metrics/{k}": v for k, v in metrics.items()})

    # ===== ハイパラ設定 + 指標を config.json と hyperparams_index.json に保存 =====
    hp_config_with_metrics = {
        "dataset_params": dataset_params,
        "model_params": model_params,
        "training_params": traing_params,
        "metrics": metrics,
    }

    # 各ハイパラディレクトリ内
    config_path = hp_dir / "config.json"
    config_path.write_text(
        json.dumps(hp_config_with_metrics, ensure_ascii=False, indent=2),
        encoding="utf-8"
    )
    print(f"Saved hyperparameter + metrics config to: {config_path}")

    # run_dir 直下のインデックス JSON を更新 (hp_dirname → hp_config_with_metrics)
    index_path = run_dir / "hyperparams_index.json"
    if index_path.exists():
        try:
            index_data = json.loads(index_path.read_text(encoding="utf-8"))
        except Exception:
            index_data = {}
    else:
        index_data = {}

    index_data[hp_dirname] = hp_config_with_metrics
    index_path.write_text(
        json.dumps(index_data, ensure_ascii=False, indent=2),
        encoding="utf-8"
    )
    print(f"Updated hyperparameter index at: {index_path}")

    print("Reservoir state visualization + richness metrics (per hyperparam) complete.")


In [ ]:


#各種のパラメータ設定
dataset_params = {"seq_start" : 400, "seq_end" : 1200, "sequence_length": 800, "slicing_size" : 1, "augmentation_factor": 0, "batch_size" : 32, "Onehot_Encoding" : None, "augmentation_mu" : 0, "augmentation_sigma" : 0, "augmentation_shift" : 1}
model_params = {"reservoir_size" : [100],"input_size" : [48], "channel_size" : [1],  "reservoir_weights_scale" : [1], "input_weights_scale" : [0.01, 1, 0.1], "spectral_radius" : [0.7, 0.9, 1.2],"reservoir_density" : [0.02], "leak_rate" : [0.1, 0.3, 0.7], "Batch_Training" : [True], "ReadOut_output_size" : [25], "Regularization_L2" : [0.01]}
training_params = {"num_epochs" : 1, "learning_rate" : 0.01, "weight_decay" : 1e-2, "testdata_ratio" : 0, "n_splits" : 5}

#それぞれのモデルパラメータ候補を辞書に格納する
model_params = model_params_candinate(model_params)

#学習データセットの設定
data_dir="./normalized_dataset_forplot/20251215_161803"
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
run_dir = Path("reservoir_vis") / timestamp
cross_val_loaders, test_loader = prepare_datasets(dataset_params, training_params, data_dir)

# wandb.init(project="uskin_test_ESN", config=config_dictionary)


#モデルパラメータの候補ごとに，総当たりしてパラメータを探索する

for each_model_params in model_params:
    
#     model = LSTMModel(each_model_params).to(device)
    model = ESN(each_model_params, training_params, dataset_params).to(device)
    model_sturcture = model_sturcture_dict(model)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr = float(training_params["learning_rate"]), weight_decay = float(training_params["weight_decay"]))
    
    config_dictionary = {
    "dataset": data_dir,
    "dataset_params" : dataset_params,
    "architecture": model.__class__.__name__,
    "model_params" : each_model_params,
    "model_sturcture" : model_sturcture,
    "traing_params" : training_params,
    "criterion" : str(criterion),
    "optimizer" : str(optimizer),
    }

    wandb.init(project="uskin_plot_test_ESN", config=config_dictionary)

    # 4. k-fold交差検証のループ
    for fold, (train_loader, val_loader) in enumerate(cross_val_loaders):
        print(f'FOLD {fold}')
        print('--------------------------------')

        # train_model(model, criterion, optimizer, train_loader, dataset_params, each_model_params, training_params)
        # visualize_reservoir_states(model, train_loader, n_units_to_plot=10)
        visualize_reservoir_states(model, train_loader, dataset_params, each_model_params, training_params, run_dir=run_dir)
        # validate_model(model, val_loader,dataset_params, each_model_params)
        model.__init__(each_model_params, training_params, dataset_params)

    print('CrossVaridation Finished')
    print('--------------------------------')
    #テストデータによる評価
    # テストデータローダーの準備

#     test_model(model, test_loader)
    
    wandb.finish()

=== TactileSequenceDataset initialized ===
root_dir     : normalized_dataset_forplot/20251215_161803
num_classes  : 25
num_samples  : 250
original T   : 1255
seq range    : [400, 1200) -> seq_len=800
n_taxels     : 16
feature_dim  : 48 (= n_taxels * 3)
class_to_idx :  {'01_table_cover': 0, '02_fur_scarf': 1, '03_washing_towel': 2, '04_carpet1': 3, '05_bubble_wrap': 4, '06_fleece_scarf': 5, '07_knit_hat1': 6, '08_body_towel1': 7, '09_body_towel2': 8, '10_carpet2': 9, '11_work_gloves': 10, '12_knit_hat2': 11, '13_toilet_mat1': 12, '14_floor_mat': 13, '15_sponge1': 14, '16_printed_tatami': 15, '17_cushion1': 16, '18_mop': 17, '19_toilet_mat2': 18, '20_fleece_sock': 19, '21_cushion': 20, '22_carpet3': 21, '23_fleece_mat': 22, '24_carpet4': 23, '25_sponge2': 24}


FOLD 0
--------------------------------
Found representative samples for 25 classes (class indices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24])
Run directory: reservoir_vis/20251215_1630
Saving visualizations under: reservoir_vis/20251215_1630/hp_35377882
[class 0] 01_table_cover | sample_idx=13 | states shape=(800, 100) | saved to reservoir_vis/20251215_1630/hp_35377882/class_00_01_table_cover.png
[class 1] 02_fur_scarf | sample_idx=6 | states shape=(800, 100) | saved to reservoir_vis/20251215_1630/hp_35377882/class_01_02_fur_scarf.png
[class 2] 03_washing_towel | sample_idx=38 | states shape=(800, 100) | saved to reservoir_vis/20251215_1630/hp_35377882/class_02_03_washing_towel.png
[class 3] 04_carpet1 | sample_idx=5 | states shape=(800, 100) | saved to reservoir_vis/20251215_1630/hp_35377882/class_03_04_carpet1.png
[class 4] 05_bubble_wrap | sample_idx=54 | states shape=(800, 100) | saved to reservoir_vis/20251215_1630/hp_35377882/clas

hp_35377882/metrics/dead_unit_ratio,▁▁▁▁▁
hp_35377882/metrics/mean_abs_correlation,▄█▁▁▃
hp_35377882/metrics/mean_dynamic_range,▁▃▄▇█
hp_35377882/metrics/mean_node_variance_across_nodes,█▃▁▄▅
hp_35377882/metrics/mean_state_norm,█▂▁▃▅
hp_35377882/metrics/mean_temporal_variance,█▃▁▃▆
hp_35377882/metrics/num_samples,▁▁▁▁▁
hp_35377882/metrics/num_units,▁▁▁▁▁
hp_35377882/metrics/participation_ratio,▄▁█▄▂
hp_35377882/metrics/saturation_ratio,▁▁▁▁▁
hp_35377882/metrics/small_dynamic_range_ratio,▁▁▁▁▁


FOLD 0
--------------------------------
Found representative samples for 25 classes (class indices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24])
Run directory: reservoir_vis/20251215_1630
Saving visualizations under: reservoir_vis/20251215_1630/hp_e044c822
[class 0] 01_table_cover | sample_idx=13 | states shape=(800, 100) | saved to reservoir_vis/20251215_1630/hp_e044c822/class_00_01_table_cover.png
[class 1] 02_fur_scarf | sample_idx=6 | states shape=(800, 100) | saved to reservoir_vis/20251215_1630/hp_e044c822/class_01_02_fur_scarf.png
[class 2] 03_washing_towel | sample_idx=38 | states shape=(800, 100) | saved to reservoir_vis/20251215_1630/hp_e044c822/class_02_03_washing_towel.png
[class 3] 04_carpet1 | sample_idx=5 | states shape=(800, 100) | saved to reservoir_vis/20251215_1630/hp_e044c822/class_03_04_carpet1.png
[class 4] 05_bubble_wrap | sample_idx=54 | states shape=(800, 100) | saved to reservoir_vis/20251215_1630/hp_e044c822/clas

hp_e044c822/metrics/dead_unit_ratio,▁▁▁▁▁
hp_e044c822/metrics/mean_abs_correlation,▅▁█▄█
hp_e044c822/metrics/mean_dynamic_range,█▃▄▅▁
hp_e044c822/metrics/mean_node_variance_across_nodes,█▂▅▃▁
hp_e044c822/metrics/mean_state_norm,█▃▄▅▁
hp_e044c822/metrics/mean_temporal_variance,█▂▅▄▁
hp_e044c822/metrics/num_samples,▁▁▁▁▁
hp_e044c822/metrics/num_units,▁▁▁▁▁
hp_e044c822/metrics/participation_ratio,▆█▁▆▅
hp_e044c822/metrics/saturation_ratio,▁▁▁▁▁
hp_e044c822/metrics/small_dynamic_range_ratio,▁▁▁▁▁


...

FOLD 0
--------------------------------
Found representative samples for 25 classes (class indices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24])
Run directory: reservoir_vis/20251215_1630
Saving visualizations under: reservoir_vis/20251215_1630/hp_6bba3b38
[class 0] 01_table_cover | sample_idx=13 | states shape=(800, 100) | saved to reservoir_vis/20251215_1630/hp_6bba3b38/class_00_01_table_cover.png
[class 1] 02_fur_scarf | sample_idx=6 | states shape=(800, 100) | saved to reservoir_vis/20251215_1630/hp_6bba3b38/class_01_02_fur_scarf.png
[class 2] 03_washing_towel | sample_idx=38 | states shape=(800, 100) | saved to reservoir_vis/20251215_1630/hp_6bba3b38/class_02_03_washing_towel.png
[class 3] 04_carpet1 | sample_idx=5 | states shape=(800, 100) | saved to reservoir_vis/20251215_1630/hp_6bba3b38/class_03_04_carpet1.png
[class 4] 05_bubble_wrap | sample_idx=54 | states shape=(800, 100) | saved to reservoir_vis/20251215_1630/hp_6bba3b38/clas

hp_6bba3b38/metrics/dead_unit_ratio,▁▁▁▁▁
hp_6bba3b38/metrics/mean_abs_correlation,▁██▅▃
hp_6bba3b38/metrics/mean_dynamic_range,▃▂██▁
hp_6bba3b38/metrics/mean_node_variance_across_nodes,▁▃█▃▁
hp_6bba3b38/metrics/mean_state_norm,▁▂█▂▁
hp_6bba3b38/metrics/mean_temporal_variance,▁▃█▃▁
hp_6bba3b38/metrics/num_samples,▁▁▁▁▁
hp_6bba3b38/metrics/num_units,▁▁▁▁▁
hp_6bba3b38/metrics/participation_ratio,▆▃▁▅█
hp_6bba3b38/metrics/saturation_ratio,▁▄█▃▂
hp_6bba3b38/metrics/small_dynamic_range_ratio,▁▁▁▁▁


FOLD 0
--------------------------------
Found representative samples for 25 classes (class indices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24])
Run directory: reservoir_vis/20251215_1630
Saving visualizations under: reservoir_vis/20251215_1630/hp_ee6b00b6
[class 0] 01_table_cover | sample_idx=13 | states shape=(800, 100) | saved to reservoir_vis/20251215_1630/hp_ee6b00b6/class_00_01_table_cover.png
[class 1] 02_fur_scarf | sample_idx=6 | states shape=(800, 100) | saved to reservoir_vis/20251215_1630/hp_ee6b00b6/class_01_02_fur_scarf.png
[class 2] 03_washing_towel | sample_idx=38 | states shape=(800, 100) | saved to reservoir_vis/20251215_1630/hp_ee6b00b6/class_02_03_washing_towel.png
[class 3] 04_carpet1 | sample_idx=5 | states shape=(800, 100) | saved to reservoir_vis/20251215_1630/hp_ee6b00b6/class_03_04_carpet1.png
[class 4] 05_bubble_wrap | sample_idx=54 | states shape=(800, 100) | saved to reservoir_vis/20251215_1630/hp_ee6b00b6/clas

hp_ee6b00b6/metrics/dead_unit_ratio,▁▁▁▁▁
hp_ee6b00b6/metrics/mean_abs_correlation,▁▆█▄▁
hp_ee6b00b6/metrics/mean_dynamic_range,▁▁▇█▆
hp_ee6b00b6/metrics/mean_node_variance_across_nodes,▁▁█▆▄
hp_ee6b00b6/metrics/mean_state_norm,▂▁█▇▄
hp_ee6b00b6/metrics/mean_temporal_variance,▁▁█▆▄
hp_ee6b00b6/metrics/num_samples,▁▁▁▁▁
hp_ee6b00b6/metrics/num_units,▁▁▁▁▁
hp_ee6b00b6/metrics/participation_ratio,▇▂▁▄█
hp_ee6b00b6/metrics/saturation_ratio,▃▁▇█▄
hp_ee6b00b6/metrics/small_dynamic_range_ratio,▁▁▁▁▁


FOLD 0
--------------------------------
Found representative samples for 25 classes (class indices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24])
Run directory: reservoir_vis/20251215_1630
Saving visualizations under: reservoir_vis/20251215_1630/hp_d663cd0e
[class 0] 01_table_cover | sample_idx=13 | states shape=(800, 100) | saved to reservoir_vis/20251215_1630/hp_d663cd0e/class_00_01_table_cover.png
[class 1] 02_fur_scarf | sample_idx=6 | states shape=(800, 100) | saved to reservoir_vis/20251215_1630/hp_d663cd0e/class_01_02_fur_scarf.png
[class 2] 03_washing_towel | sample_idx=38 | states shape=(800, 100) | saved to reservoir_vis/20251215_1630/hp_d663cd0e/class_02_03_washing_towel.png
[class 3] 04_carpet1 | sample_idx=5 | states shape=(800, 100) | saved to reservoir_vis/20251215_1630/hp_d663cd0e/class_03_04_carpet1.png
[class 4] 05_bubble_wrap | sample_idx=54 | states shape=(800, 100) | saved to reservoir_vis/20251215_1630/hp_d663cd0e/clas